In [ ]:
AOI = "MULTIPOLYGON (((-110.49367408906883 39.1005200348037, -110.20673076923077 39.1005200348037, -110.16573886639677 38.41732165423689, -110.43901821862349 38.41732165423689, -110.49367408906883 39.1005200348037)))"
START_DATE = "2016-05-01"
END_DATE = "2023-06-30"

REQUEST_ID = '0'

In [ ]:
import os
import json
import time
import shutil
import rasterio
import urllib.request
import rasterio.mask as riomask
import shapely
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pylab as plt
from rasterio.merge import merge

from utils import crop_raster, merge_tiles, stitch_tiles, get_tiles, transform_crs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
NAME = "Landcover"
NB_USER = os.getenv('NB_USER')
BASE = os.getcwd()
DATA_DIR = f"/home/{NB_USER}/work/jupyter/Landcover"

LOAD_DIR = os.path.join(DATA_DIR, "satellite_imagery")
RESULTS_DIR = os.path.join(DATA_DIR, "results/landcover")

In [ ]:
default_crs = 'EPSG:4326'
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(
    gpd.GeoSeries([polygon]),
    columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")
aoi = gpd.read_file(aoi_filename)

In [ ]:
sentinel_tiles_path = os.path.join(DATA_DIR, "sentinel2grid.geojson")
sentinel_tiles_path = "sentinel2grid.geojson"
tile_idx = get_tiles(aoi_filename, sentinel_tiles_path)

start_year = int(START_DATE.split("-")[0])
end_year = int(END_DATE.split("-")[0])
date_range = range(2017, 2022)

if end_year in date_range:
    year = end_year
elif start_year in date_range:
    year = start_year
else:
    year = max(list(date_range))

In [ ]:
year

In [ ]:
files = []
data_dir = f"{DATA_DIR}/landcover_dataset"
os.makedirs(data_dir, exist_ok=True)

for tile_i in tile_idx.tileID:
    tile_i = tile_i if len(tile_i) < 4 else tile_i[:3]
    tile_url = f"https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc{year}/{tile_i}_{year}0101-{year+1}0101.tif"
    path = f"{data_dir}/{os.path.basename(tile_url)}"
    if not os.path.exists(path):
        urllib.request.urlretrieve(tile_url, path)
    files.append(path)
    
files

In [ ]:
class_names = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built Area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland"
}

In [ ]:
if len(files) > 1:
    try:
        raster_path = merge_tiles(
            files, os.path.join(data_dir, "aoi_raster.tif"))
    except Exception:
        raster_path = stitch_tiles(
            files, os.path.join(data_dir, "aoi_raster.tif"))
else:
    raster_path = files[0]

In [ ]:
raster_path = crop_raster(
    raster_path,
    aoi_filename,
    raster_path.replace(".tif", "_crop.tif"),
    additional_meta={
        "REQUEST_ID": REQUEST_ID,
        "START_DATE": f"{year}-01-01",
        "END_DATE": f"{year+1}-01-01",
        "NAME": "Landcover classification"})

with rasterio.open(raster_path) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

In [ ]:
NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0,0,0,1)

labels = []

for label, name in enumerate(class_names):
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    class_color = ",".join(list(map(lambda x: str(int(x)), colors[label][:-1] * 255)))
    if class_area != 0:
        labels.append({
            "color": class_color, 
            "name": class_names[name],
            "area": class_area
        })
    else:
        name = "Other" if name not in class_names.keys() else class_names[name]
        labels.append({
            "color": class_color, 
            "name": name,
            "area": 0.0
        })
labels = json.dumps(labels)
labels

In [ ]:
os.makedirs(f"/home/{NB_USER}/work/results/landcover", exist_ok=True)
tile_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"

NUM_CLASSES = 11
nodata = 0
mask = mask.astype(bool)
scaled = img.astype(np.float32) / NUM_CLASSES
scaled = (plt.cm.jet(scaled)[:,:,:-1] * 255)
scaled[mask[:,:,np.newaxis] & (scaled==0)] += 1
scaled = np.clip(scaled, 0, 255).astype(np.uint8)
# Set pixels with invalid pixels to new nodata value
scaled[~mask] = nodata
# Set pixels with background class(0) to new nodata value
scaled[img==0] = nodata

profile.update(
    count=3,
    nodata=nodata,
    compress='lzw'
)
colored_tif = f"/home/{NB_USER}/work/results/landcover/{tile_name}"

with rasterio.open(colored_tif, 'w', **profile) as dst:
    dst.update_tags(start_date=START_DATE, 
                    end_date=END_DATE, 
                    request_id=REQUEST_ID,
                    labels=labels,
                    name=NAME)
    for i in range(scaled.shape[-1]):
        dst.write(scaled[:,:,i], indexes=i+1)

In [ ]:
os.remove(raster_path)
os.remove(aoi_filename)